In [1]:
from model import Generator
from PIL import Image
from torch.autograd import Variable
from utils import *
from easydict import EasyDict as edict
import glob
import cv2
from torchvision.utils import save_image as imwrite

In [2]:
#Check if gpu is available
torch.cuda.is_available()

True

In [10]:
args = edict()

args.testImages = '/home/trojan/Desktop/Image restoration/Homeworks/HW4/dataset/Dehazing_datasets/OutdoorTestHazy/rescaled'
args.outDir = '/home/trojan/Desktop/Image restoration/Homeworks/HW4/result/testing2/Outdoor'
args.gpu = True 
args.model = '/home/trojan/Desktop/Image restoration/Homeworks/HW4/result/Net_dehaze/model/model_lastest.pt'

In [11]:
#Basic Settings
if args.gpu == 0:
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
elif args.gpu == 1:
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [12]:
#Check cuda device
device = 'cuda' if torch.cuda.is_available() and args.gpu else 'cpu'
print (device)

cuda


In [13]:
def test(args):
    my_model = Generator()
    #my_model.cuda()
    my_model.load_state_dict(torch.load(args.model))
    my_model.eval()

    output_dir = args.outDir
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)

    img_dir = args.testImages

    test_images = []
    for file_name in os.listdir(img_dir):
        test_images.append(os.path.join(img_dir,file_name))
        
    image = Image.open(test_images[1]).convert('RGB')
    #image = cv2.imread(img) 
    width, height = image.size
    scale = 32
    image = image.resize((width // scale * scale, height // scale * scale))
    with torch.no_grad():
        image = rgb_to_tensor(image)
        image = image.unsqueeze(0)
        #image = Variable(image.cuda())
        output = my_model(image)
    output = tensor_to_rgb(output)
    out = Image.fromarray(np.uint8(output), mode='RGB')
    out = out.resize((width, height), resample=Image.BICUBIC)
    output_path = args.outDir
    #cv2.imwrite(output_path + '{}.png'.format(n), out)
    0
    print('Image saved at ' + output_path)
    
    '''n = 0
    for img in test_images:
        image = Image.open(img).convert('RGB')
        #image = cv2.imread(img) 
        width, height = image.size
        scale = 32
        image = image.resize((width // scale * scale, height // scale * scale))
        with torch.no_grad():
            image = rgb_to_tensor(image)
            image = image.unsqueeze(0)
            #image = Variable(image.cuda())
            output = my_model(image)
        output = tensor_to_rgb(output)
        out = Image.fromarray(np.uint8(output), mode='RGB')
        out = out.resize((width, height), resample=Image.BICUBIC)
        output_path = os.path.join(args.outDir, os.path.basename(img_dir))
        #cv2.imwrite(output_path + '{}.png'.format(n), out)
        out.save(output_path + '{}.png'.format(n))
        print('One image saved at ' + output_path)
        n+=1'''

In [14]:
if __name__ == '__main__':
    test(args)

Image saved at /home/trojan/Desktop/Image restoration/Homeworks/HW4/result/testing2/Outdoor
